In [1]:
%load_ext sql

In [2]:
%sql sqlite:///library.db

'Connected: @library.db'

## Creating Tables


In [152]:
%%sql

CREATE TABLE Author (
    aid INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(15) NOT NULL,
    last_name VARCHAR(15),
    birthdate TEXT,
    UNIQUE(first_name, last_name, birthdate),
    CHECK (birthdate IS DATE(birthdate))
)

 * sqlite:///library.db
Done.


[]

In [153]:
%%sql

CREATE TABLE Item (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title VARCHAR(20) NOT NULL,
    year INTEGER,
    type VARCHAR(15) NOT NULL,
    aid INTEGER,
    FOREIGN KEY (aid) REFERENCES Author(aid),
    CHECK (type = 'print' OR type ='digital' OR type ='magazine' OR type ='journal' OR type ='cd' 
           OR type ='record' OR type ='dvd' OR type ='game'),
    CHECK (year BETWEEN 0 AND 9999)
)

 * sqlite:///library.db
Done.


[]

In [154]:
%%sql

CREATE TABLE Customer (
    cid INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(15) NOT NULL,
    last_name VARCHAR(15) NOT NULL,
    birthdate TEXT NOT NULL,
    UNIQUE(first_name, last_name, birthdate),
    CHECK (birthdate IS DATE(birthdate))
)

 * sqlite:///library.db
Done.


[]

In [155]:
%%sql

CREATE TABLE Borrowed (
    id INTEGER,
    cid INTEGER,
    borrow_date TEXT DEFAULT (DATE('now', 'localtime')),
    due_date TEXT DEFAULT (DATE('now', '+10 days')),
    PRIMARY KEY (id, cid),
    FOREIGN KEY (id) REFERENCES Item(id),
    FOREIGN KEY (cid) REFERENCES Customer(cid),
    CHECK (borrow_date IS DATE(borrow_date)),
    CHECK (due_date IS DATE(due_date))
)

 * sqlite:///library.db
Done.


[]

In [156]:
%%sql

CREATE TABLE Fine (
    id INTEGER,
    cid INTEGER,
    amount REAL DEFAULT 1.00,
    PRIMARY KEY (id, cid),
    FOREIGN KEY (id) REFERENCES Item(id)  ,
    FOREIGN KEY (cid) REFERENCES Customer(cid)
)

 * sqlite:///library.db
Done.


[]

In [157]:
%%sql

CREATE TABLE Event (
    eid INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(20) NOT NULL,
    e_type VARCHAR(20) NOT NULL,
    audience VARCHAR(20) DEFAULT 'all', 
    CHECK (audience = 'all' OR audience = 'child' OR audience = 'adolescent' OR audience ='adult'),
    CHECK (e_type = 'club' OR e_type = 'signing' OR e_type = 'art' OR e_type = 'screening')
)

 * sqlite:///library.db
Done.


[]

In [158]:
%%sql

CREATE TABLE Room (
    room_number INTEGER PRIMARY KEY AUTOINCREMENT,
    capacity INTEGER NOT NULL,
    CHECK (capacity > 0)
)

 * sqlite:///library.db
Done.


[]

In [159]:
%%sql

CREATE TABLE Room_Booking (
    eid INTEGER,
    room_number INTEGER,
    date TEXT NOT NULL,
    start_time TEXT NOT NULL,
    end_time TEXT NOT NULL,
    PRIMARY KEY (eid, room_number),
    FOREIGN KEY (eid) REFERENCES Event(eid),
    FOREIGN KEY (room_number) REFERENCES Room(room_number),
    CHECK (end_time > start_time),
    CHECK (start_time IS strftime('%H:%M',start_time)),
    CHECK (end_time IS strftime('%H:%M',end_time)),
    CHECK (date IS DATE(date))
)

 * sqlite:///library.db
Done.


[]

In [160]:
%%sql

CREATE TABLE Personnel (
    pid INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(15) NOT NULL,
    last_name VARCHAR(15) NOT NULL,
    birthdate TEXT NOT NULL,
    role VARCHAR(15) NOT NULL,
    join_date TEXT NOT NULL,
    salary REAL NOT NULL,
    CHECK (role = 'supervisor' OR role = 'volunteer'),
    CHECK (salary >= 0),
    CHECK (birthdate IS DATE(birthdate)),
    UNIQUE(first_name, last_name, birthdate)
)

 * sqlite:///library.db
Done.


[]

In [161]:
%%sql

CREATE TABLE Future_Item (
    fid INTEGER PRIMARY KEY AUTOINCREMENT,
    title VARCHAR(20) NOT NULL,
    year INTEGER,
    type VARCHAR(15) NOT NULL,
    aid INTEGER,
    FOREIGN KEY (aid) REFERENCES Author(aid),
    CHECK (type = 'print' OR type ='digital' OR type ='magazine' OR type ='journal' OR type ='cd' 
       OR type ='record' OR type ='dvd' OR type ='game'),
    CHECK (year BETWEEN 0 AND 9999)
)

 * sqlite:///library.db
Done.


[]

## Creating Triggers

In [162]:
%%sql

CREATE TRIGGER check_room_booking_overlap
BEFORE INSERT ON Room_Booking
WHEN EXISTS (
    SELECT * FROM Room_Booking
    WHERE eid = NEW.eid
    AND room_number = NEW.room_number
    AND date = NEW.date
    AND start_time <= NEW.start_time
    AND end_time >= NEW.end_time
)
BEGIN
    SELECT RAISE (FAIL, 'Overlapping events');
END;

CREATE TRIGGER check_room_booking_event
BEFORE INSERT ON Room_Booking
WHEN NOT EXISTS (
    SELECT * FROM Event
    WHERE eid = NEW.eid
)
BEGIN
    SELECT RAISE (FAIL, 'No such event');
END;

CREATE TRIGGER check_room_booking_room
BEFORE INSERT ON Room_Booking
WHEN NOT EXISTS (
    SELECT * FROM Room
    WHERE room_number = NEW.room_number
)
BEGIN
    SELECT RAISE (FAIL, 'No such room');
END;

 * sqlite:///library.db
Done.
Done.
Done.


[]

In [163]:
%%sql

CREATE TRIGGER check_existing_customer
BEFORE INSERT ON Borrowed
WHEN NOT EXISTS (
    SELECT * FROM Customer
    WHERE cid = NEW.cid
)
BEGIN
    SELECT RAISE (FAIL, 'No such customer');
END;

CREATE TRIGGER check_existing_item
BEFORE INSERT ON Borrowed
WHEN NOT EXISTS (
    SELECT * FROM Item
    WHERE id = NEW.id
)
BEGIN
    SELECT RAISE (FAIL, 'No such item');
END;

 * sqlite:///library.db
Done.
Done.


[]

## Inserting Values

In [164]:
%%sql

INSERT INTO Author (first_name, last_name, birthdate)
VALUES ('Stephen', 'King', '1947-09-21');

INSERT INTO Author (first_name, last_name, birthdate)
VALUES ('Elton', 'John', '1947-03-25');

INSERT INTO Author (first_name, last_name, birthdate)
VALUES ('Markus', 'Persson', '1979-06-01');

INSERT INTO Author (first_name, birthdate)
VALUES ('Dr.Seuss', '1904-03-02');

INSERT INTO Author (first_name, last_name, birthdate)
VALUES ('Frank', 'Ocean', '1987-10-28');

INSERT INTO Author (first_name, last_name, birthdate)
VALUES ('Peter', 'Norvig', '1956-12-14');

INSERT INTO Author (first_name, last_name, birthdate)
VALUES ('Anna', 'Wintour', '1949-01-03');

INSERT INTO Author (first_name, last_name, birthdate)
VALUES ('Eazy', 'E', '1964-09-07');

INSERT INTO Author (first_name, last_name, birthdate)
VALUES ('Charles', 'Dickens', '1812-02-07');

INSERT INTO Author (first_name, last_name, birthdate)
VALUES ('William', 'Shakespeare', '1564-04-01');



 * sqlite:///library.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [165]:
%%sql

INSERT INTO Room (capacity)
VALUES (2);
INSERT INTO Room (capacity)
VALUES (10);
INSERT INTO Room (capacity)
VALUES (5);
INSERT INTO Room (capacity)
VALUES (20);
INSERT INTO Room (capacity)
VALUES (5);
INSERT INTO Room (capacity)
VALUES (3);
INSERT INTO Room (capacity)
VALUES (15);
INSERT INTO Room (capacity)
VALUES (15);
INSERT INTO Room (capacity)
VALUES (20);
INSERT INTO Room (capacity)
VALUES (5);

 * sqlite:///library.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [166]:
%%sql

INSERT INTO Event (name, e_type, audience)
VALUES ('Inaugural Book Club Meeting', 'club', 'all');

INSERT INTO Event (name, e_type, audience)
VALUES ('Disney movie Screening', 'screening', 'all');

INSERT INTO Event (name, e_type, audience)
VALUES ('Horror movie Screening', 'screening', 'adult');

INSERT INTO Event (name, e_type, audience)
VALUES ('Books and Wine', 'club', 'adult');

INSERT INTO Event (name, e_type, audience)
VALUES ('Stephen King Book Signing', 'signing', 'adolescent');

INSERT INTO Event (name, e_type, audience)
VALUES ('Pixar movie Screening', 'screening', 'all');

INSERT INTO Event (name, e_type, audience)
VALUES ('Paint and Wine', 'art', 'adult');

INSERT INTO Event (name, e_type, audience)
VALUES ('Intro to drawing', 'art', 'child');

INSERT INTO Event (name, e_type, audience)
VALUES ('Dr.Seuss Book signing', 'signing', 'child');

INSERT INTO Event (name, e_type, audience)
VALUES ('Family Movie Night', 'screening', 'all');


 * sqlite:///library.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [167]:
%%sql

INSERT INTO Item (title, year, type, aid)
VALUES ('The Shining', 1977, 'print', 1);

INSERT INTO Item (title, year, type, aid)
VALUES ('The Stand', 1990, 'print', 1);

INSERT INTO Item (title, year, type, aid)
VALUES ('The Cat in the Hat', 1957, 'print', 4);

INSERT INTO Item (title, year, type, aid)
VALUES ('How the Grinch Stole Christmas!', 1957, 'print', 4);

INSERT INTO Item (title, year, type, aid)
VALUES ('Elton John', 1970, 'cd', 2);

INSERT INTO Item (title, year, type, aid)
VALUES ('Channel Orange', 2012, 'cd', 5);

INSERT INTO Item (title, year, type, aid)
VALUES ('Vogue', 1999, 'magazine', 7);

INSERT INTO Item (title, year, type, aid)
VALUES ('Google Hybrid Approach to Research', 2012, 'journal', 6);

INSERT INTO Item (title, year, type, aid)
VALUES ('Oliver Twist', 1838, 'print', 9);

INSERT INTO Item (title, year, type, aid)
VALUES ('A Christmas Carol', 1843, 'print', 9);


 * sqlite:///library.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [168]:
%%sql

INSERT INTO Future_Item (title, year, type, aid)
VALUES('Blonde', 2016, 'cd', 5);

INSERT INTO Future_Item (title, year, type, aid)
VALUES('It', 1986, 'print', 1);

INSERT INTO Future_Item (title, year, type, aid)
VALUES('Holly', 2023, 'print', 1);

INSERT INTO Future_Item (title, year, type, aid)
VALUES('Great Eggs and Ham', 1960, 'print', 4);

INSERT INTO Future_Item (title, year, type, aid)
VALUES('Fox in Socks', 1965, 'print', 4);

INSERT INTO Future_Item (title, year, type, aid)
VALUES('Bleak House', 1853, 'print', 9);

INSERT INTO Future_Item (title, year, type, aid)
VALUES('David Copperfield', 1850, 'print', 9);

INSERT INTO Future_Item (title, year, type, aid)
VALUES('Shambles', 2013, 'game', 3);

INSERT INTO Future_Item (title, year, type, aid)
VALUES('AI an Intelligent Approach', 2009, 'digital', 6);

INSERT INTO Future_Item (title, year, type, aid)
VALUES('Nostalgia Ultra', 2011, 'record', 5);


 * sqlite:///library.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [169]:
%%sql

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('Justin', 'Xie', '2001-10-23');

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('John', 'Smith', '1991-01-28');

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('Sally', 'Doe', '2015-11-30');

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('Simon', 'Rief', '1961-04-01');

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('Alan', 'Lee', '2001-06-06');

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('Owen', 'George', '2004-12-31');

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('Natalie', 'Ngo', '1998-09-22');

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('Lois', 'Lane', '1972-01-04');

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('Mike', 'Ike', '2010-02-16');

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('Chris', 'Chan', '1989-03-29');

INSERT INTO Customer (first_name, last_name, birthdate)
VALUES ('Christine', 'Chan', '1991-01-23');

 * sqlite:///library.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [170]:
%%sql

INSERT INTO Personnel (first_name, last_name, birthdate, role, join_date, salary)
VALUES ('Andrew', 'Mcdonald', '2004-12-12', 'volunteer', '2022-03-12', 0);

INSERT INTO Personnel (first_name, last_name, birthdate, role, join_date, salary)
VALUES ('Dalauan', 'Sparrow', '2000-09-17', 'supervisor', '2022-03-12', 50000);

INSERT INTO Personnel (first_name, last_name, birthdate, role, join_date, salary)
VALUES ('Mike', 'Andrews', '2002-09-12', 'volunteer', '2023-04-16', 0);

INSERT INTO Personnel (first_name, last_name, birthdate, role, join_date, salary)
VALUES ('Rory', 'Mcdonald', '1998-01-09', 'volunteer', '2022-03-12', 0);

INSERT INTO Personnel (first_name, last_name, birthdate, role, join_date, salary)
VALUES ('Kirk', 'Robles', '2001-03-12', 'supervisor', '2023-02-18', 45000);

INSERT INTO Personnel (first_name, last_name, birthdate, role, join_date, salary)
VALUES ('Andrew', 'Lee', '2006-08-24', 'volunteer', '2023-06-25', 0);

INSERT INTO Personnel (first_name, last_name, birthdate, role, join_date, salary)
VALUES ('Sarah', 'Chow', '2002-05-30', 'volunteer', '2023-06-24', 0);

INSERT INTO Personnel (first_name, last_name, birthdate, role, join_date, salary)
VALUES ('Sally', 'Smith', '1984-07-23', 'supervisor', '2001-04-25', 65000);

INSERT INTO Personnel (first_name, last_name, birthdate, role, join_date, salary)
VALUES ('Jean', 'Dawson', '2004-12-12', 'volunteer', '2021-04-25', 0);

INSERT INTO Personnel (first_name, last_name, birthdate, role, join_date, salary)
VALUES ('Frank', 'Ocean', '2004-12-12', 'volunteer', '2019-06-06', 0);

 * sqlite:///library.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [171]:
%%sql

INSERT INTO Room_Booking (eid, room_number, date, start_time, end_time)
VALUES (1, 2, '2013-01-01', "04:15", "05:15");

INSERT INTO Room_Booking (eid, room_number, date, start_time, end_time)
VALUES (3, 4, '2013-03-01', "07:15", "08:15");

INSERT INTO Room_Booking (eid, room_number, date, start_time, end_time)
VALUES (3, 3, '2021-01-01', "10:00", "12:00");

INSERT INTO Room_Booking (eid, room_number, date, start_time, end_time)
VALUES (2, 8, '2014-02-01', "04:15", "05:15");

INSERT INTO Room_Booking (eid, room_number, date, start_time, end_time)
VALUES (8, 9, '2018-01-09', "15:30", "18:15");

INSERT INTO Room_Booking (eid, room_number, date, start_time, end_time)
VALUES (4, 6, '2023-01-01', "14:15", "15:15");

INSERT INTO Room_Booking (eid, room_number, date, start_time, end_time)
VALUES (7, 1, '2023-01-02', "07:30", "09:45");

INSERT INTO Room_Booking (eid, room_number, date, start_time, end_time)
VALUES (9, 7, '2017-11-11', "04:15", "05:15");

INSERT INTO Room_Booking (eid, room_number, date, start_time, end_time)
VALUES (1, 4, '2022-08-03', "04:15", "05:15");

INSERT INTO Room_Booking (eid, room_number, date, start_time, end_time)
VALUES (2, 3, '2018-01-09', "04:15", "05:15");

 * sqlite:///library.db
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [172]:
%%sql

INSERT INTO Borrowed (id, cid)
VALUES (1, 6),
    (2, 6),
    (3, 7),
    (4, 7),
    (5, 8),
    (6, 8),
    (7, 9),
    (8, 9),
    (9, 10),
    (10, 10);


 * sqlite:///library.db
10 rows affected.


[]

In [173]:
%%sql

INSERT INTO Fine (id, cid)
VALUES (1, 1),
    (2, 1),
    (3, 2),
    (4, 2),
    (5, 3),
    (6, 3),
    (7, 4),
    (8, 4),
    (9, 5),
    (10, 5);

 * sqlite:///library.db
10 rows affected.


[]

In [151]:
'''
%%sql

DROP TABLE Author;
DROP TABLE Item;
DROP TABLE Customer;
DROP TABLE Borrowed;
DROP TABLE Fine;
DROP TABLE Event;
DROP TABLE Room;
DROP TABLE Room_Booking;
DROP TABLE Personnel;
DROP TABLE Future_Item;
'''

 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [174]:
%%sql

SELECT * FROM Personnel

 * sqlite:///library.db
Done.


pid,first_name,last_name,birthdate,role,join_date,salary
1,Andrew,Mcdonald,2004-12-12,volunteer,2022-03-12,0.0
2,Dalauan,Sparrow,2000-09-17,supervisor,2022-03-12,50000.0
3,Mike,Andrews,2002-09-12,volunteer,2023-04-16,0.0
4,Rory,Mcdonald,1998-01-09,volunteer,2022-03-12,0.0
5,Kirk,Robles,2001-03-12,supervisor,2023-02-18,45000.0
6,Andrew,Lee,2006-08-24,volunteer,2023-06-25,0.0
7,Sarah,Chow,2002-05-30,volunteer,2023-06-24,0.0
8,Sally,Smith,1984-07-23,supervisor,2001-04-25,65000.0
9,Jean,Dawson,2004-12-12,volunteer,2021-04-25,0.0
10,Frank,Ocean,2004-12-12,volunteer,2019-06-06,0.0
